In [1]:
#API Key laden
#OpenAI-API-Schlüssel sicher aus gespeicherten Colab-Benutzerdaten in Notebook schreiben
from google.colab import userdata
OPENAI_API_KEY = userdata.get('apikey_11')

In [ ]:
#Installation
#LangChain-Integrationspaket für OpenAI installieren, um OpenAI-Modelle direkt in LangChain zu nutzen
# ! -> in Shell ausgeführt
!pip install langchain_openai

In [ ]:
#Installation
#LangChain-Paket mit Schnittstellen zu externen Tools wie z.b. PyPDFLoader
# ! -> in Shell ausgeführt
!pip install langchain_community

In [4]:
#Imports
#Text-Splitter aus LangChain-Bibliothek
#Teilt lange Texte in Chunks, um Embeddings zu berechnen
#Geht Rekursiv vor: zuerst Absätze, dann Sätze, dann Zeichen
from langchain_text_splitters import RecursiveCharacterTextSplitter

#OpenAIEmbeddings ist die Schnittstelle um OpenAI-Modelle für Embeddings zu nutzen
#ChatOpenAI ist die Schnittstelle zu den OpenAI-Chatmodellen
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

#FAISS ist eine Vektor-Datenbank von Meta, die Embeddings speichert
#FAISS läuft lokal
from langchain_community.vectorstores import FAISS

#PyPDFLoader übersetzt PDFs zu LangChain-Dokumenten zur Weiterverarbeitung
from langchain_community.document_loaders import PyPDFLoader

#Chain ist die kombinierte Pipeline in LangChain
#RetrievalQA umfasst den vollständigen Ablauf des RAGs und übernimmt somit das Zusammenspiel zwischen den Komponenten
from langchain.chains import RetrievalQA

In [ ]:
#Installation
#Python-Bibliothek zum Arbeiten mit PDF-Dateien
# ! -> in Shell ausgeführt
!pip install pypdf

In [21]:
#externe Quelle (PDF)
import requests

# URL, wo das PDF abliegt
url = "https://www.h-kronenberg.ch/obst_wein_schnaps_essig/Anleitungen/Wein%20Herstellung.pdf"

#Hier wird ein loader für die PDF erstellt
loader = PyPDFLoader(url)
#Der loader extrahiert den Text aus dem PDF aufgeteilt nach Seiten und speichert das Ergebnis in docs.
#docs ist eine Liste von LangChain-Dokumenten für jeweils jede Seite
#Jedes Dokument enthält den Inhalt und Metadaten für die jeweilige Seite
docs = loader.load()


In [7]:
#Chunking

#chunk_size -> wie groß sollen Chunks sein
#chunk_overlap -> wie viel sollen die Chunks sich überschneiden, um keinen Kontext zu verlieren
# hier: Jeder Abschnitt überlappt sich mit dem vorherigen um 30 Zeichen
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
#Liste docs wird mit denkleineren Chunks überschrieben
docs = splitter.split_documents(docs)

In [ ]:
#Installation

#Python-Paket von Pypl
#FAISS = Facebook AI Similarity Search
#--> Bibliothek von Meta zur Vektoren- und Ähnlichkeitssuche
# ! -> in Shell ausgeführt
!pip install faiss-cpu

In [9]:
#Embedding + Vektordatenbank (FAISS)

#OpenAIEmbeddings ist die Schnittstelle von LangChain zu OpenAI-Embedding-Modellen
#openai_api_key --> API-Schlüssel, damit LangChain die OpenAI-API nutzen darf
#model --> das Modell, das OpenAI für Embeddings bereitstellt; günstiger und schneller als text-embedding-3-large
#embeddings kann beliebige Texte in semantische Vektoren umwandeln
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
#Jedes Dokument aus Liste docs wird mit Embeddings in Embedding-Vektor umgewandelt
#Embedding-Vektoren werden in FAISS abgelegt
#vectorstore enthält die Embedding-Vektoren
vectorstore = FAISS.from_documents(docs, embeddings)

In [10]:
#Retriever

#aus vectorstore wird ein Retriever gemacht (as_retriever), als Schnittstelle für die semantische Suche
#retiever kann von LangChain in QA-Chains verwendet werden
#search_type="similarity" --> wie soll gesucht werden
#search_kwargs={"k": 3} --> gib 3 Chunks zurück
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [11]:
#LLM

#ChatOpenAI --> Schnittstelle von LangChain zu OpenAI
#openai_api_key --> Schlüssel, damit LangChain auf Modell zugreifen kann
#model ist das Sprachmodell das bei OpenAI verwendet wird
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [12]:
#Pipeline: Retrieval + QA-Chain

#QA_Chain ist die komplette Pipeline zusammengefasse
#RetrievalQA ist fertige Chain in LangChain, die Retrieval und LLM kombiniert
#from_chain_type baut die QA-Pipeline auf, damit nicht jeder Schritt manuell zusammengestellt werden muss
#Aus dem vorherigen Schitten wird das LLM und der Retriever übergeben
#chain_type definiert, wie gefundenen Chunks an LLM übergeben werden
#--> "stuff" -> alle relevanten Dokumente werden einfach in das LLM gegeben
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [13]:
#Testanfrage
query = "Auf was muss man besonders bei der Herstellung von Wein achten?"

#die Pipeline wird aufgerufen mit der query und gibt die Antowrt zurück
response = qa_chain.invoke(query)

print(response)

{'query': 'Auf was muss man besonders bei der Herstellung von Wein achten?', 'result': 'Bei der Herstellung von Wein aus Weintrauben ist besonders darauf zu achten, dass die Trauben sorgfältig behandelt werden, um die Qualität des Weines zu gewährleisten.'}
